### Steps for downloading GBIF data for another species for your portfolio post

#### Step 1: choose a terrestrial species that migrates (and find its scientific/Latin name)

#### Step 2: make an account on [gbif.org](<https://www.gbif.org/)>)

#### Step 3: Install the pygbif package in your python environment

If you're on a Mac, just run this code chunk:

In [ ]:
%% bash
pip install pygbif

If you're on a PC, open Git Bash in the Terminal at the bottom of the Jupyter notebook, and enter this: 

`pip install pygbif`

#### Step 4: load the packages you'll need

In [31]:
import pathlib
from pathlib import Path
import os
import pygbif.occurrences as occ
import pygbif.species as species
import getpass
import glob
import time
import pandas as pd
import zipfile

#### Step 5: make a folder for your GBIF data
You'll need to modify this code to match your folder structure:

In [12]:
### create data directory
crane_dir = os.path.join(

    ### home directory
    pathlib.Path.home(),

    ### folder with assignments for this class
    'OneDrive - UCB-O365',
    'Earth-Data-Science',
    'Migration-Assignment',
    'Data',
    'purple_martins')

### make the directory
os.makedirs(crane_dir, exist_ok = True)

### define directory for gbif data
gbif_dir = os.path.join(crane_dir, 'gbif')

### make the directory
os.makedirs(gbif_dir, exist_ok = True)

In [13]:
gbif_dir

'C:\\Users\\matth\\OneDrive - UCB-O365\\Earth-Data-Science\\Migration-Assignment\\Data\\purple_martins\\gbif'

#### Step 6: link GBIF to your Jupyter notebook
- Change to `reset = True` the first time you run it. When you run it, you'll be prompted to enter your GBIF username, password, and associated email address
- Then change it back to `reset = False` so you can re-run the chunk without having to reconnect to GBIF
- You don’t need to modify this code chunk in any other way


In [ ]:
####--------------------------####
#### DO NOT MODIFY THIS CODE! ####
####--------------------------####
# This code ASKS for your credentials 
# and saves it for the rest of the session.
# NEVER put your credentials into your code!!!!

# GBIF needs a username, password, and email 
# All 3 need to match the account
reset = False

# Request and store username
if (not ('GBIF_USER'  in os.environ)) or reset:
    os.environ['GBIF_USER'] = input('GBIF username:')

# Securely request and store password
if (not ('GBIF_PWD'  in os.environ)) or reset:
    os.environ['GBIF_PWD'] = getpass.getpass('GBIF password:')
    
# Request and store account email address
if (not ('GBIF_EMAIL'  in os.environ)) or reset:
    os.environ['GBIF_EMAIL'] = input('GBIF email:')

#### Step 7: Get the species key for your species from GBIF using the `name_backbone` command
You'll need to modify this code with the scientific name of your species

In [20]:
### grab the species info
backbone = species.name_backbone(name = 'Progne subis')

### check it out
backbone



{'usageKey': 2489129,
 'scientificName': 'Progne subis (Linnaeus, 1758)',
 'canonicalName': 'Progne subis',
 'rank': 'SPECIES',
 'status': 'ACCEPTED',
 'confidence': 99,
 'matchType': 'EXACT',
 'kingdom': 'Animalia',
 'phylum': 'Chordata',
 'order': 'Passeriformes',
 'family': 'Hirundinidae',
 'genus': 'Progne',
 'species': 'Progne subis',
 'kingdomKey': 1,
 'phylumKey': 44,
 'classKey': 212,
 'orderKey': 729,
 'familyKey': 5248,
 'genusKey': 2489127,
 'speciesKey': 2489129,
 'class': 'Aves'}

In [21]:
### pull out the species key
species_key = backbone['usageKey']

### check it out
species_key

2489129

#### Step 8: Run this code to download the GBIF data once
It might take several minutes for the data to download

Modify this code to update the year if you want data for a different year, and to use the correct data_dir for your file structure (my code says "gbif_dir")

In [ ]:
# Only download once
### set file name for download
gbif_pattern = os.path.join(gbif_dir, '*.csv')

### double check that there isn't already a file that matches this pattern.
### if it already exists, skip the whole conditional
### and go straight to the line: gbif_path = glob(gbif_pattern)[0]
if not glob.glob(gbif_pattern):

    ### only submit a download request to GBIF once
    ### if GBIF_DOWNLOAD_KEY is not defined in our environment, make the download request
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:

        ### submit a query to GBIF
        gbif_query = occ.download([

            ### add your species key here
            f"speciesKey = {species_key}",

            ### filter out results that are missing coordinates
            "hasCoordinate = True",

            ### choose a year to include
            "year = 2024",
        ])
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

    # Wait for the download to build
    download_key = os.environ['GBIF_DOWNLOAD_KEY']

    ### use the occurrence command module in pygbif to get the metadata
    wait = occ.download_meta(download_key)['status']

    ### check if the status of the download = "SUCCEEDED"
    ### wait and loop through until it finishes
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(download_key)['status']

        ### don't want to re-query the API in the loop too frequently
        time.sleep(5)

    # Download GBIF data when it's ready
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=gbif_dir)

    # Unzip GBIF data using the zipfile package
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path=gbif_dir)

# Find the extracted .csv file path (take the first result)
gbif_path = glob.glob(gbif_pattern)[0]

The GBIF occurrence data should now be in a folder on your computer, saved as a .csv file. You can open it using the code structure we've used before to open .csv files using pandas.

In [35]:
gbif_df = pd.read_csv(gbif_path, delimiter='\t')
gbif_df.head()

C:\Users\matth\AppData\Local\Temp\ipykernel_18964\854206072.py:1: DtypeWarning: Columns (10,14,40,41,43,48) have mixed types. Specify dtype option on import or set low_memory=False.
  gbif_df = pd.read_csv(gbif_path, delimiter='\t')


,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,5321904726,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS2306035416,Animalia,Chordata,Aves,Passeriformes,Hirundinidae,Progne,Progne subis,...,NaN,NaN,CC_BY_4_0,NaN,obsr186024,NaN,NaN,2025-10-08T14:29:54.530Z,NaN,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_CONCE...
1,5511104165,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS2196430608,Animalia,Chordata,Aves,Passeriformes,Hirundinidae,Progne,Progne subis,...,NaN,NaN,CC_BY_4_0,NaN,obsr1799659,NaN,NaN,2025-10-08T14:29:54.530Z,NaN,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_CONCE...
2,5325050824,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS2272960429,Animalia,Chordata,Aves,Passeriformes,Hirundinidae,Progne,Progne subis,...,NaN,NaN,CC_BY_4_0,NaN,obsr1020433,NaN,NaN,2025-10-08T14:29:54.530Z,NaN,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_CONCE...
3,5592401814,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS2332296033,Animalia,Chordata,Aves,Passeriformes,Hirundinidae,Progne,Progne subis,...,NaN,NaN,CC_BY_4_0,NaN,obsr443672,NaN,NaN,2025-10-08T14:29:54.530Z,NaN,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_CONCE...
4,5380403789,4fa7b334-ce0d-4e88-aaae-2e0c138d049e,URN:catalog:CLO:EBIRD:OBS1979138449,Animalia,Chordata,Aves,Passeriformes,Hirundinidae,Progne,Progne subis,...,NaN,NaN,CC_BY_4_0,NaN,obsr712111,NaN,NaN,2025-10-08T14:29:54.530Z,NaN,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_CONCE...


In [ ]:
%store gbif_df